# I - Theoretical Calculations

We have the following Hamiltonian for the circuit : 

$$
H = \frac {C_T} 2 \dot{{\phi_T}}^2 + \frac {C_R} 2 \dot{{\phi_R}}^2 + \frac {C_C} 2 (\dot{{\phi_T}} -\dot{{\phi_R}})^2 + \frac 1 {2L}{\phi_R}^2 - E_J \cos\bigg(\frac {\phi_T} {\phi_0}\bigg)
$$

With $\phi_0 = \frac \hbar {2e}$, and the variables indexed by $T$ represent the Transmon part of the circuits, the ones indexed bu $R$ the Resonator ones, and by $C$ the Coupling ones.

We have the following capacitance matrix :
$$
\mathcal{C} =
\begin{pmatrix}
C_T + C_C & C_C \\
C_C & C_R + C_C
\end{pmatrix}
$$

This leads us to define :
$$E_{C_T} := \frac {e^2} {2(C_T+C_C)}$$
$$E_{C_R} := \frac {e^2} {2(C_R+C_C)}$$
$$E_{L} := \frac {{\phi_0}^2} {2L}$$

And :

$$
\begin{align}
n_T := \frac 1 {2e} (C_T + C_C) \dot{\phi}_T \;\;\; &; \;\;\; \varphi_T := \frac {\phi_T} {\phi_0}\\
n_R := \frac 1 {2e} (C_R + C_C) \dot{\phi}_R \;\;\; &; \;\;\; \varphi_R := \frac {\phi_R} {\phi_0}
\end{align}
$$
$$$$

$$$$

This leads to :
$$
\begin{align}
H = & \ 4E_{C_T}{n_T}^2 - E_J \cos(\varphi_T) \\
&+ 4E_{C_R}{n_R}^2 + \frac {E_L} 2 \varphi_R \\ 
& -4e^2 \frac{C_C}{C_T C_R} n_T n_R
\end{align}
$$

i.e.

$$
\begin{align}
H = & \ H_T \\
&+ H_R \\ 
&+ H_C
\end{align}
$$

Let's define the coupling energy :
$$E_{coupling} := e^2 \frac{C_C}{C_T C_R}$$

Finally, let's write this Hamiltonian as an operator. We get:

$$
\begin{align}
\hat H = & \ \bigg[4E_{C_T}{\hat n_T}^2 - E_J \cos(\hat\varphi_T)\bigg] \otimes {\mathbb{I}} \\
&+ \mathbb{I} \otimes \bigg[4E_{C_R}{\hat n_R}^2 + \frac {E_L} 2 \hat\varphi_R^2\bigg] \\ 
& -4E_{coupling} \ \hat n_T \otimes \hat n_R
\end{align}
$$

And if we recall that $\hat n_R = \frac{\hat a - \hat a^{\dagger}} {2i\varphi_{ZPF}}$
where $\varphi_{ZPF} = \bigg(\frac {2E_C} {E_L} \bigg)^{\frac 1 4}$, we get the expression of $\hat H$ using the creation and destruction operators, and the charge operators only, meaning :

$$
\begin{align}
\hat H = & \ \bigg[4E_{C_T}{\hat n_T}^2 - E_J \cos(\hat\varphi_T)\bigg] \otimes {\mathbb{I}} \\
&+ \mathbb{I} \otimes \left(\hbar \omega (\hat a^{\dagger} \hat a) + \frac 1 2\right) \\ 
& -4E_{coupling} \ \hat n_T \otimes \left(\frac{\hat a - \hat a^{\dagger}} {2i\varphi_{ZPF}}\right)
\end{align}
$$

# II - Implementation using SCQubits

In [80]:
import scqubits as sc
import numpy as np

In [81]:
def translate(L): return L-L[0] #changes all values by the first value of the list

In [82]:
def eigenvalues_sc(ECT, ECR, ECoup, EJ, EL, transmon_trunc=31, resonator_trunc=20):

    transmon = sc.Transmon(
        EJ=EJ,
        EC=ECT,
        ng=0.0,
        ncut=transmon_trunc,
        truncated_dim=4
    )

    resonator = sc.Oscillator(
        E_osc=np.sqrt(8*ECR*EL),
    )

    hilbertspace = sc.HilbertSpace([transmon, resonator])

    phizpf = ((2*ECR) / EL) ** (1 / 4)

    n_resonator = (resonator.annihilation_operator() - resonator.creation_operator()) / (2j * phizpf)

    hilbertspace.add_interaction(
        g=-4*ECoup,
        op1=(transmon.n_operator, transmon),
        op2=(n_resonator, resonator)
    )

    return hilbertspace.eigensys(evals_count=4)[0]

In [83]:
evals_sc = translate(eigenvalues_sc(1,1,1,1,1))

# III - Implementation using QuTiP

In [84]:
import qutip as qt

In [85]:
def hamiltonian_qt(ECT, ECR, ECoup, EJ, EL, transmon_trunc=31, resonator_trunc=40) :

    ## Transmon Hamiltonian
    charge = qt.charge(transmon_trunc//2, -transmon_trunc//2)
    cos_phi = qt.tunneling(2 * transmon_trunc//2 + 1)

    HT = 4 * ECT * charge * charge - EJ / 2 * cos_phi

    ## Resonator Hamiltonian
    creation = qt.create(resonator_trunc)  # Creation and destruction operators
    destruction = qt.destroy(resonator_trunc)

    HR = (
        np.sqrt(8 * ECR * EL) * (creation * destruction + 1 / 2)
    )

    ## Coupling Hamiltonian

    phizpf = ((2*ECR) / EL) ** (1 / 4)
    n_R = (destruction - creation) / (2j * phizpf)

    HC = -4*ECoup * qt.tensor(charge,n_R)


    ## Final Hamiltonian :
    return qt.tensor(HT, qt.qeye(resonator_trunc)) + qt.tensor(qt.qeye(2 * transmon_trunc//2 + 1),HR) + HC

In [86]:
evals_qt = translate(hamiltonian_qt(1,1,1,1,1).eigenenergies())[:4]

# IV - Implementation using Numpy

In [87]:
import numpy.linalg as alg

In [88]:
def hamiltonian_np(ECT, ECR, ECoup, EJ, EL, transmon_trunc=31, resonator_trunc=40):

    ## Transmon Hamiltonian

    dim_transmon = 2*transmon_trunc//2 + 1
    charge = np.zeros((dim_transmon, dim_transmon))
    cos_phi = np.zeros((dim_transmon, dim_transmon))

    for i in range(dim_transmon-1):
        cos_phi[i, i + 1] = 1
        cos_phi[i + 1, i] = 1
        charge[i, i] = i - dim_transmon//2

    charge[-1, -1] = dim_transmon//2

    HT = 4 * ECT * charge**2 - EJ / 2 * cos_phi


    ## Resonator Hamiltonian

    destruction = np.zeros((resonator_trunc, resonator_trunc))

    for n in range(1, resonator_trunc):
        destruction[n - 1, n] = np.sqrt(n)

    creation = destruction.T 

    HR =  np.sqrt(8 * ECR * EL) * (
        np.dot(creation, destruction) + np.eye(resonator_trunc) * (1 / 2)
    )


    ## Coupling Hamiltonian

    phizpf = ((2*ECR) / EL) ** (1 / 4)
    n_R = (destruction - creation) / (2j * phizpf)

    HC = -4*ECoup * np.kron(charge,n_R)


    ## Final Hamiltonian

    return np.kron(HT, np.eye(resonator_trunc)) + np.kron(np.eye(dim_transmon), HR) + HC



In [89]:
evals_np = translate(np.sort(alg.eigvals(hamiltonian_np(1,1,1,1,1))))[:4]

In [90]:
print(evals_sc,'\n', evals_qt,'\n', evals_qt)

[0.         2.62916495 3.21348713 3.36296111] 
 [0.         2.62882932 3.21257187 3.36230896] 
 [0.         2.62882932 3.21257187 3.36230896]
